## AIHub Preprocessing Runtime Calculator

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [3]:
import re
import os
import kss
import ray
import json
import time
import datetime
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain
from difflib import get_close_matches

In [2]:
pwd

'c:\\Users\\MinSeok\\Documents\\AIHUB'

### AIHUB 일반상식

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=116&topMenu=100&aihubDataSe=ty&dataSetSn=106)

In [ ]:
from sentence_segmentation import preprocessing_text
from data_preprocessing import make_pro_post_txt_file_path_list
from reading_data import reading_txt

In [ ]:
from sentence_segmentation import preprocessing_text
from data_preprocessing import make_pro_post_txt_file_path_list
from data_preprocessing import merge_and_deduplicate_corpus_txt
from reading_data import reading_txt

In [18]:
pro_corpus_path = "AIHUB_corpus/exploration/general_common_sense_pro/AIHUB_general_common_sense_" + "*.txt"
pro_total_corpus_path_list, post_total_corpus_path_list = make_pro_post_txt_file_path_list(pro_corpus_path)

In [19]:
len(pro_total_corpus_path_list)

362

In [20]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "source"

reading_txt(pro_coprus_file, line_length, data_type)

아동학대범죄처벌법에서는 피해아동 및 학대행위자는 피해아동보호명령사건에 대하여 각자 보조인을 선임할 수 있으며, 피해아동 및 학대행위자의 법정대리인·배우자·직계친족·형제자매, 아동보호전문기관의 상담원과 그 기관장 및 피해자변호사는 보조인이 될 수 있다(제48조). 또 피해아동의 경우 1. 피해아동에게 신체적·정신적 장애가 의심되는 경우, 2. 빈곤이나 그 밖의 사유로 보조인을 선임할 수 없는 경우, 3. 그 밖에 판사가 보조인이 필요하다고 인정하는 경우 중의 어느 하나에 해당하는 경우에 법원의 직권 또는 피해아동 또는 피해아동의 법정대리인·직계친족·형제자매, 아동보호전문기관의 상담원과 그 기관장의 신청이 있으면 변호사를 학대행위자의 보조인으로 선정할 수 있다(제49조 제1항). 학대행위자가 형소법상 국선변호인 선정대상(제33조 제1항) 중의 어느 하나에 해당하는 경우에도 법원은 직권으로 변호사를 학대행위자의 보조인으로 선정할 수 있다.(동조 제2항)



In [21]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "preprocessing"

reading_txt(pro_coprus_file, line_length, data_type)

아동학대범죄처벌법에서는 피해아동 및 학대행위자는 피해아동보호명령사건에 대하여 각자 보조인을 선임할 수 있으며, 피해아동 및 학대행위자의 법정대리인 배우자 직계친족 형제자매, 아동보호전문기관의 상담원과 그 기관장 및 피해자변호사는 보조인이 될 수 있다.

또 피해아동의 경우 1. 피해아동에게 신체적 정신적 장애가 의심되는 경우, 2. 빈곤이나 그 밖의 사유로 보조인을 선임할 수 없는 경우, 3. 그 밖에 판사가 보조인이 필요하다고 인정하는 경우 중의 어느 하나에 해당하는 경우에 법원의 직권 또는 피해아동 또는 피해아동의 법정대리인 직계친족 형제자매, 아동보호전문기관의 상담원과 그 기관장의 신청이 있으면 변호사를 학대행위자의 보조인으로 선정할 수 있다.



In [22]:
ray.init(num_cpus = 4)

@ray.remote
def ray_preprocessing_text(source):

    preprocessing_sentence_list = preprocessing_text(source)

    return preprocessing_sentence_list

2023-07-25 18:16:43,392	INFO worker.py:1625 -- Started a local Ray instance.


In [32]:
def preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list):
    
    progress_length = len(pro_total_corpus_path_list)
    process_num = 10    

    for pro, post in zip(pro_total_corpus_path_list, post_total_corpus_path_list):

        sentence_list = []

        with open(pro, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines() 
            nested_lines_num = len(lines) // process_num
            for i in range(nested_lines_num - 1):
                start_line = process_num * i
                end_line = process_num * (i+1)
                futures = [ray_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
                results = ray.get(futures)

                if i == nested_lines_num - 2:
                    futures = [ray_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                    results = ray.get(futures)

                sentences = list(chain.from_iterable(results))
                sentence_list.append(sentences)

        sentence_list = list(chain.from_iterable(sentence_list))

        with open(post, 'a', encoding='utf-8') as fp:
            fp.write("\n".join(sentence_list))

### Preprocessing Runtime

In [33]:
def calculate_runtime_source_list_10000_size():
    
    start = time.time()

    preprocessing_corpus_txt([pro_total_corpus_path_list[0]], [pro_total_corpus_path_list[0]])

    end = time.time()

    runtime = end - start

    return runtime

In [118]:
def make_excel_group_list(excel_list, corpus_list):
    
    excel_group_list = []

    for corpus in corpus_list:
        excel = get_close_matches(corpus, excel_list, n = 34)
        excel_group_list.append(excel)

    excel_group_list[2] = ['source_file_by_batch\\document_summary_text_train.xlsx', 
                        'source_file_by_batch\\document_summary_text_valid.xlsx']
    excel_group_list[7] = ['source_file_by_batch\\machine_reading.xlsx']

    return excel_group_list

In [123]:
def make_preprocessing_runtime_df(excel_group_list, corpus_list, runtime_unit):

    preprocessing_runtime_df = pd.DataFrame({'File':[0], 'Length of Source List':[0],
                                    'Preprocessing Runtime':[0]})

    preprocessing_total_length = 0 

    for idx, (excel_group, corpus) in enumerate(zip(excel_group_list, corpus_list)):

        preprocessing_length = 0

        for excel_file in excel_group:
            df = pd.read_excel(excel_file, engine='openpyxl') 
            preprocessing_length += df['Length of Source List'].sum()
            preprocessing_total_length += df['Length of Source List'].sum()
        
        runtime_second = runtime_unit * (preprocessing_length / 10000)
        runtime = str(datetime.timedelta(seconds = runtime_second))
        preprocessing_runtime_df.loc[idx] = [corpus, preprocessing_length, runtime]
        
    runtime_total_second = runtime_unit * (preprocessing_total_length / 10000)
    runtime_total = str(datetime.timedelta(seconds = runtime_total_second)).split(".")[0]    
    preprocessing_runtime_df.loc[len(corpus_list)] = ["total", preprocessing_total_length, runtime_total]
    
    preprocessing_runtime_df.to_excel("aihub_preprocessing_runtime_df.xlsx", index=False)

    return preprocessing_runtime_df

In [ ]:
runtime_unit = calculate_runtime_source_list_10000_size()

In [ ]:
print(runtime_unit)

31.116286277770996


In [40]:
ray.shutdown()

In [125]:
source_file_by_batch_excels = glob("source_file_by_batch/*.xlsx")
corpus_list = ['automatic_patent_classification_data', 
               'british_korean_data_on_patents_in_major_countries_linked_to_industrial_information',
               'document_summary_text', 'general_common_sense', 'korean_corpus_data_based_on_large_scale_purchase_books',
               'legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data',
               'machine_reading_data_for_administrative_documents', 'machine_reading',
               'news_article_machine_reading_data', 'reading_books_by_machine', 'summary_and_report_generation_data',
               'summary_of_book_materials', 'summary_of_thesis_materials', 'web_data_based_korean_corpus_data']

excel_group_list = make_excel_group_list(source_file_by_batch_excels, corpus_list)

In [126]:
print(excel_group_list)

[['source_file_by_batch\\automatic_patent_classification_data_valid.xlsx', 'source_file_by_batch\\automatic_patent_classification_data_train.xlsx'], ['source_file_by_batch\\british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_valid.xlsx', 'source_file_by_batch\\british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_train.xlsx'], ['source_file_by_batch\\document_summary_text_train.xlsx', 'source_file_by_batch\\document_summary_text_valid.xlsx'], ['source_file_by_batch\\general_common_sense.xlsx'], ['source_file_by_batch\\korean_corpus_data_based_on_large_scale_purchase_books.xlsx'], ['source_file_by_batch\\legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_valid.xlsx', 'source_file_by_batch\\legal_regulations_(such_as_terms_and_conditions_of_judgment)_text_analysis_data_train.xlsx'], ['source_file_by_batch\\machine_reading_data_for_administrative_documents_valid.xlsx', 'source_file_by_batch\\machine_r

In [127]:
preprocessing_runtime_df = make_preprocessing_runtime_df(excel_group_list, corpus_list, runtime_unit)

In [128]:
preprocessing_runtime_df

,File,Length of Source List,Preprocessing Runtime
0,automatic_patent_classification_data,496850,0:25:46.012684
1,british_korean_data_on_patents_in_major_countr...,359999,0:18:40.183194
2,document_summary_text,2741982,2:22:12.029688
3,general_common_sense,73516,0:03:48.754490
4,korean_corpus_data_based_on_large_scale_purcha...,887511,0:46:01.604635
5,legal_regulations_(such_as_terms_and_condition...,34699,0:01:47.970402
6,machine_reading_data_for_administrative_documents,469480,0:24:20.847408
7,machine_reading,101844,0:05:16.900706
8,news_article_machine_reading_data,360060,0:18:40.373004
9,reading_books_by_machine,237500,0:12:19.011799
